In [ ]:
import math
import numpy as np
import pandas as pd
# import h5py
import matplotlib.pyplot as plt
# import scipy
# from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
# from cnn_utils import *
import pydicom
%matplotlib inline
import glob
import os
import cv2
from numpy import random
from tensorflow.keras.constraints import unit_norm, max_norm
# from IPython.core.interactiveshell import InteractiveShell

# InteractiveShell.ast_node_interactivity = "all"

In [ ]:
empt_arr = np.empty((585*4,150,150, 40), 'uint8')

row_no = 0
channel = 0

path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train"

folders = ["FLAIR","T2w","T1w","T1wCE"]

# for folder in folder:
for folder in folders:
    print("Folder started -----------------------------:", folder)
    for i in range(0,1011):
        a = '00000' + str(i)

        path_flair = sorted(glob.glob(os.path.join(path, a[-5:] , folder, '*')), 
                       key = lambda x : int(x[:-4].split("-")[-1]))
        leth =  len(path_flair)
#         print("length of folder" + str(len(path_flair)))
        
        k1 = len(glob.glob(os.path.join(path, a[-5:] , "FLAIR", '*')))
        k2 = len(glob.glob(os.path.join(path, a[-5:] , "T2w", '*')))
        k3 = len(glob.glob(os.path.join(path, a[-5:] , "T1w", '*')))
        k4 = len(glob.glob(os.path.join(path, a[-5:] , "T1w", '*')))

        if k1 == 0 and k2 == 0 and k3 == 0 and k4 == 0:
            continue
#         print("folder number: " + str(a[-5:]))

        for i in range(len(path_flair)):
                if (i+1)%7 == 0 and i > 0: 
#                     print("image number---",i)
                    dicom_arr = pydicom.dcmread(path_flair[i])
                    dicom_arr = dicom_arr.pixel_array
                    dicom_arr = dicom_arr[100:400,100:400]
                    dicom_arr = cv2.resize(dicom_arr, (0, 0), fx = 0.5, fy = 0.5)

                    dicom_arr = dicom_arr - np.min(dicom_arr)
                    if np.max(dicom_arr) != 0:
                        dicom_arr = dicom_arr / (np.max(dicom_arr) - np.min(dicom_arr))
                        dicom_arr = dicom_arr * 255
                        
#                         print("image shape:", dicom_arr.shape)
                        a,b = dicom_arr.shape
                        if a < 150 or b < 150:
                            empt_arr[row_no,0:a,0:b,channel] = dicom_arr 
                        else:
                            empt_arr[row_no,:,:,channel] = dicom_arr
#                         print("channel number---", channel)
                        if channel == 39:
                            break
                        channel += 1
                    
        channel = 0
#         print("channel reset----------------------", channel)
        row_no += 1
#         print("Row number----------------------", row_no)
        
                             
                

In [ ]:
# ../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv
# ../input/train-labels ../input/train-labels/train_labels.csv
Train_Y = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv', encoding = 'utf-8')
# Train_Y

In [ ]:
Train_Y1 = list(Train_Y["MGMT_value"])
Train_Y2 = Train_Y1*4

In [ ]:
Train_Y3 = np.array(Train_Y2)
# Train_Y3.shape

In [ ]:
# from tensorflow.keras.constraints import unit_norm, max_norm

# cons = 5.
# init = 'HeUniform'
def forward_propagation(H,W,C):
    tf.compat.v1.set_random_seed(3) 
    model = tf.keras.Sequential(name="RSNA")
    model.add(tf.keras.layers.InputLayer(input_shape=(H,W,C)))
    model.add(tf.keras.layers.Conv2D(filters = 16, kernel_size = (5,5),kernel_constraint = max_norm(5.), kernel_regularizer=regularizers.l2(0.001),strides=(1, 1), padding='same',activation = 'relu',kernel_initializer='HeUniform'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1,1), padding='same'))
    model.add(tf.keras.layers.Dropout(0.5))
    #Layer2
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3),kernel_constraint=max_norm(5.), kernel_regularizer=regularizers.l2(0.001),strides=(1, 1), padding='same',activation = 'relu',kernel_initializer='HeUniform'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1,1), padding='same'))
    #Dropout
    model.add(tf.keras.layers.Dropout(0.5))
    
    #FLATTEN
    model.add(tf.keras.layers.Flatten()) 
#               kernel_regularizer=regularizers.l2(0.001)
    #FC OR DENSELY CONNECTED
    model.add(tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.0001),kernel_constraint=max_norm(10.),kernel_initializer='HeUniform'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.0001),kernel_constraint=max_norm(10.),kernel_initializer='HeUniform'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(2, activation='softmax',kernel_initializer='he_normal'))  
    
    #compile
#     model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#     model.fit(X_train,Y_train,epochs=10,batch_size=32,validation_split=0.1,verbose = 1)
#     model.output_shape
    return model

In [ ]:

H, W, C = empt_arr.shape[1:]
H, W, C

In [ ]:
#compile
from tensorflow.python.keras import regularizers
# H, W, C = X_train.shape[1:]     
model = forward_propagation(H, W, C)
model.compile(loss='categorical_crossentropy',optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
from tensorflow.keras.utils import to_categorical
Y_train4 = to_categorical(Train_Y3, num_classes=2)
Y_train4.shape, empt_arr.shape

In [ ]:
# from tensorflow.python.keras import regularizers
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU
# with tpu_strategy.scope():
#     model = forward_propagation(H, W, C)
#     model.compile(loss='categorical_crossentropy',optimizer='adam',
#               metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

# # train model normally
# # model.fit(training_dataset, epochs=EPOCHS, steps_per_epoch=…)

In [ ]:
history = model.fit(empt_arr, Y_train4,epochs=30,batch_size=32,validation_split=0.1,verbose = 1)

In [ ]:
# model.summary()

In [ ]:
# history.history.keys()

In [ ]:
# loss = history.history['loss'][1:]
# val_loss = history.history['val_loss'][1:]
# epochs = range(1, len(loss) + 1)
# plt.plot(epochs, loss, 'y', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [ ]:
# # Split the train and the validation set for the fitting
# from sklearn.model_selection import train_test_split
# X_train1, X_val, Y_train1, Y_val = train_test_split(empt_arr, Y_train4, test_size = 0.1, random_state=2)
# print("x_train shape",X_train1.shape)
# print("x_val shape",X_val.shape)
# print("y_train shape",Y_train1.shape)
# print("y_val shape",Y_val.shape)

In [ ]:


# # Predict the values from the validation dataset
# Y_pred = model.predict(X_val)
# # Convert predictions classes to one hot vectors 
# Y_pred_classes = tf.math.argmax(Y_pred, axis=1)
# # Convert validation observations to one hot vectors
# Y_true = tf.math.argmax(Y_val,axis = 1) 
# # compute the confusion matrix
# confusion_mtx = tf.math.confusion_matrix(Y_true, Y_pred_classes) 

# m = tf.keras.metrics.Recall()
# m.update_state(Y_true, Y_pred_classes)
# m.result()


In [ ]:
# ds = pydicom.dcmread(os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test","00001/FLAIR/Image-100.dcm"))
# ds = ds.pixel_array
# ds = ds[100:400,100:400]
# half = cv2.resize(ds, (0, 0), fx = 0.5, fy = 0.5)
# half.shape,plt.imshow(half, cmap=plt.cm.bone)

In [ ]:
Test_X = np.empty((87*1,150,150, 40), 'uint8')

row_no = 0
channel = 0
# ../input/rsna-miccai-brain-tumor-radiogenomic-classification/test
path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

folders = ["FLAIR"] #,"T2w","T1w","T1wCE"

# for folder in folder:
for folder in folders:
    print("Folder started -----------------------------:", folder)
    for i in range(0,1011):
        a = '00000' + str(i)

        path_flair = sorted(glob.glob(os.path.join(path, a[-5:] , folder, '*')), 
                       key = lambda x : int(x[:-4].split("-")[-1]))
        leth =  len(path_flair)
#         print("length of folder" + str(len(path_flair)))
        
        k1 = len(glob.glob(os.path.join(path, a[-5:] , "FLAIR", '*')))
        k2 = len(glob.glob(os.path.join(path, a[-5:] , "T2w", '*')))
        k3 = len(glob.glob(os.path.join(path, a[-5:] , "T1w", '*')))
        k4 = len(glob.glob(os.path.join(path, a[-5:] , "T1w", '*')))

        if k1 == 0 and k2 == 0 and k3 == 0 and k4 == 0:
            continue
#         print("folder number: " + str(a[-5:]))

        for i in range(len(path_flair)):
                if (i+1)%7 == 0 and i > 0: 
#                     print("image number---",i)
                    dicom_arr = pydicom.dcmread(path_flair[i])
                    dicom_arr = dicom_arr.pixel_array
                    dicom_arr = dicom_arr[100:400,100:400]
                    dicom_arr = cv2.resize(dicom_arr, (0, 0), fx = 0.5, fy = 0.5)

                    dicom_arr = dicom_arr - np.min(dicom_arr)
                    if np.max(dicom_arr) != 0:
                        dicom_arr = dicom_arr / (np.max(dicom_arr) - np.min(dicom_arr))
                        dicom_arr = dicom_arr * 255
                        
#                         print("image shape:", dicom_arr.shape)
                        a,b = dicom_arr.shape
                        if a < 150 or b < 150:
                            Test_X[row_no,0:a,0:b,channel] = dicom_arr 
                        else:
                            Test_X[row_no,:,:,channel] = dicom_arr
#                         print("channel number---", channel)
                        if channel == 39:
                            break
                        channel += 1
                    
        channel = 0
#         print("channel reset----------------------", channel)
        row_no += 1
#         print("Row number----------------------", row_no)
        
                   
#  ds = pydicom.dcmread(os.path.join("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train","00000/FLAIR/Image-100.dcm"))
# ds = ds.pixel_array
# ds = ds[100:400,100:400]
# half = cv2.resize(ds, (0, 0), fx = 0.5, fy = 0.5)                  
                

In [ ]:
ID = []
path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

folders = ["FLAIR"]

# for folder in folder:
for folder in folders:
    
    for i in range(0,1007):
        a = '00000' + str(i)

        path_flair = sorted(glob.glob(os.path.join(path, a[-5:] , folder, '*')), 
                       key = lambda x : int(x[:-4].split("-")[-1]))
        leth =  len(path_flair)
        
        
        k1 = len(glob.glob(os.path.join(path, a[-5:] , "FLAIR", '*')))
        k2 = len(glob.glob(os.path.join(path, a[-5:] , "T2w", '*')))
        k3 = len(glob.glob(os.path.join(path, a[-5:] , "T1w", '*')))
        k4 = len(glob.glob(os.path.join(path, a[-5:] , "T1w", '*')))

        if k1 == 0 and k2 == 0 and k3 == 0 and k4 == 0:
            continue
        ID.append(str(a[-5:]))
#         print("folder number: " + str(a[-5:]))
len(ID)

In [ ]:
# ID

In [ ]:
# Test_X.shape

In [ ]:
# Predict
# tf.compat.v1.enable_eager_execution()
y_pred = model.predict(Test_X)

In [ ]:
# y_pred.shape

In [ ]:
# Y_predicted = tf.math.argmax(y_pred, axis=1)
Y_predicted = np.max(y_pred, axis = 1)

In [ ]:
# Y_predicted

In [ ]:
my_submission = pd.DataFrame({'BraTS21ID': ID, 'MGMT_value': Y_predicted})
# my_submission = pd.read_csv(f"{data_directory}/sample_submission.csv", dtype=str)pwd
# ..
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [ ]:
my_submission